# Desarrollo de Programa para Estimación de Costos y Verificación de Material

La empresa de energía del Quindío (EDEQ) requiere desarrollar un programa en Python para que sus usuarios obtengan un costo estimado de la construcción de una red eléctrica y determinen la disponibilidad del material necesario para la ejecución del proyecto.

### Parámetros de Entrada del Usuario:

1. **Tipo de Red:**
   - a. Red cubierta (ACSR SEMI-AISLADO Ecológico)
   - b. Red desnuda (ACSR desnudo)

2. **Calibre del Cable:**
   - a. 2 AWG
   - b. 1/0 AWG
   - c. 2/0 AWG
   - d. 4/0 AWG

3. **Kilómetros de Red de la Instalación:**
   - (Puede ser un número entero o decimal)

### Cálculos a Realizar:

1. **Costo Total del Proyecto:**
   - Se calculará en pesos.

2. **Verificación de Material:**
   - El programa determinará si hay suficiente material en la bodega de la empresa para ejecutar la totalidad del proyecto.

### Elementos Necesarios para la Instalación de Cada Tipo de Red:

**Red Cubierta:**
   - a. 1 separador de red compacta cada 8 metros.
   - b. 1 poste cada 40 metros.
   - c. 1 sistema de puesta a tierra (SPT) cada 250 metros.

**Red Desnuda:**
   - a. Sin separadores de red.
   - b. 1 poste cada 70 metros.
   - c. 1 sistema de puesta a tierra (SPT) cada 250 metros.

### Elementos Necesarios para Cada Poste:

   - a. 2 retenidas directas a tierra de media tensión (M.T) 13.2 kV.
   - b. 1 estructura para soportar el cable (ACSR Desnudo o ACSR Semiaislado).
   - c. 1 poste de concreto de 14 metros de altura y 750 kgf de carga de rotura (14m X 750Kg).

### Consideraciones Importantes:

1. Las redes a instalar son expansiones de red (redes nuevas) y no requieren el retiro de elementos existentes.
2. El nivel de tensión de trabajo es media tensión (M.T) según la clasificación RETIE, específicamente (13.2 kV).
3. Las redes a instalar son trifásicas (3 hilos por cada tramo de red).
4. Se proporciona una tabla en el libro de Excel "Relacion_MO_MAT.xlsx", que expresa los materiales y sus cantidades necesarios para llevar a cabo una actividad de mano de obra.

In [14]:
import pandas as pd

# Función para calcular el costo total del proyecto
def calcular_costo_total(relacion_mo_mat, precios_mano_obra, precios_material, existencias_material):
    costo_total = 0
    material_insuficiente = []

    for index, row in relacion_mo_mat.iterrows():
        codigo_mo = row['CODIGO_MO']
        codigo_material = row['CODIGO_MATERIAL']
        cantidad_unitaria = row['CANTIDAD_UNITARIA_MATERIAL']

        # Obtener el valor unitario de la mano de obra
        mano_obra_match = precios_mano_obra[precios_mano_obra['CODIGO_MO'] == codigo_mo]
        if not mano_obra_match.empty:
            valor_mano_obra = mano_obra_match['VALOR_UNITARIO'].values[0]
        else:
            return costo_total, f"No se encontró valor de mano de obra para {row['DESCRIPCION_MATERIAL']} ({codigo_mo})"

        # Obtener el valor unitario del material
        material_match = precios_material[precios_material['CODIGO_MATERIAL'] == codigo_material]
        if not material_match.empty:
            valor_material = material_match['VALOR_UNITARIO'].values[0]
        else:
            return costo_total, f"No se encontró valor de material para {row['DESCRIPCION_MATERIAL']} ({codigo_material})"

        # Calcular el costo parcial y sumarlo al costo total
        costo_parcial = cantidad_unitaria * (valor_mano_obra + valor_material)
        costo_total += costo_parcial

        # Verificar la existencia de material en la bodega
        existencia_match = existencias_material[existencias_material['CODIGO_MATERIAL'] == codigo_material]
        if not existencia_match.empty:
            cantidad_existente = existencia_match['CANTIDAD_EXISTENCIAS'].values[0]
            if cantidad_unitaria > cantidad_existente:
                material_insuficiente.append(row['DESCRIPCION_MATERIAL'])

    if material_insuficiente:
        return costo_total, f"Material insuficiente para los siguientes materiales: {', '.join(material_insuficiente)}"
    else:
        return costo_total, "Proyecto viable"

# Función para solicitar una entrada válida
def solicitar_entrada(opciones_validas, mensaje):
    while True:
        entrada = input(mensaje).upper()
        if entrada in opciones_validas:
            return entrada
        else:
            print("Ingrese un valor válido.")

# Función principal
def main():
    while True:
        try:
            # Solicitar al usuario los parámetros de entrada
            tipo_red = solicitar_entrada(['A', 'B'], "Ingrese el tipo de red (a para Red cubierta, b para Red desnuda): ")
            calibre_cable = solicitar_entrada(['A', 'B', 'C', 'D'], "Ingrese el calibre del cable (a para 2 AWG, b para 1/0 AWG, c para 2/0 AWG, d para 4/0 AWG): ")
            km_red = float(input("Ingrese la longitud de la red en kilómetros: "))

            # Cargar los datos desde los archivos Excel
            relacion_mo_mat = pd.read_excel("Relacion_MO_MAT.xlsx")
            precios_mano_obra = pd.read_excel("Precios_Mano_Obra.xlsx")
            precios_material = pd.read_excel("Precios_Material.xlsx")
            existencias_material = pd.read_excel("Existencias_Material.xlsx")

            # Calcular el costo total del proyecto y verificar la existencia de material
            costo_total, mensaje = calcular_costo_total(relacion_mo_mat, precios_mano_obra, precios_material, existencias_material)

            # Mostrar resultados al usuario
            print(f"\nCosto total del proyecto: ${costo_total:.2f}")
            print(mensaje)

            # Preguntar al usuario si desea realizar otro cálculo
            continuar = solicitar_entrada(['S', 'N'], "¿Desea realizar otro cálculo? (s/n): ")
            if continuar != 'S':
                break
        except ValueError:
            print("Ingrese valores válidos.")
        except Exception as e:
            print(f"Error: {str(e)}")

# Llamada a la función principal
main()


Ingrese el tipo de red (a para Red cubierta, b para Red desnuda): a
Ingrese el calibre del cable (a para 2 AWG, b para 1/0 AWG, c para 2/0 AWG, d para 4/0 AWG): g
Ingrese un valor válido.
Ingrese el calibre del cable (a para 2 AWG, b para 1/0 AWG, c para 2/0 AWG, d para 4/0 AWG): a
Ingrese la longitud de la red en kilómetros: 12.36

Costo total del proyecto: $66058486.87
No se encontró valor de mano de obra para ESPIGO RECTO PARA EXTREMO DE POSTE 13.2 KV (3_28-T)
¿Desea realizar otro cálculo? (s/n): n
